In [0]:
import pandas as pd
import numpy as np
import datetime
import plotly.express as px

In [0]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [0]:
import pickle

# 1. Reading data and EDA
source: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page
## 1.1. Yellow Datatrips for Jan 2023

In [0]:
df_jan = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
df_feb = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

df_jan.head()

In [0]:
df_jan.describe(include='all')

In [0]:
df_jan.info()

## Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [0]:
df_jan.shape

## Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [0]:
df_jan["duration"] = df_jan["tpep_dropoff_datetime"] - df_jan["tpep_pickup_datetime"]
df_jan[["tpep_pickup_datetime", "tpep_dropoff_datetime", "duration" ]].head()

In [0]:
df_jan["duration"] = df_jan["duration"].apply(lambda td: td.total_seconds() / 60)
df_jan[["tpep_pickup_datetime", "tpep_dropoff_datetime", "duration" ]].head()

In [0]:
df_jan["duration"].std()

## Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [0]:
#fig = px.histogram(df_jan, x="duration")
#fig.show()

In [0]:
df_jan["duration"].describe(percentiles=[0.8, 0.85, 0.9, 0.95, 0.98, 0.99])

In [0]:
rows_old = df_jan.shape[0]
df_jan = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]
rows_new = df_jan.shape[0]
round(100.00*rows_new/rows_old)

## Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [0]:
df_jan.columns

In [0]:
categorical = ['PULocationID', 'DOLocationID']

df_jan[categorical] = df_jan[categorical].astype(str)

In [0]:
train_dicts = df_jan[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

X_train

In [0]:
len(dv.feature_names_)

## Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters
- Calculate the RMSE of the model on the training data
What's the RMSE on train?

In [0]:
target = 'duration'
y_train = df_jan[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
np.sqrt(mean_squared_error(y_train, y_pred))

In [0]:
#with open('models/lin_reg.bin', 'wb') as f_out:
#    pickle.dump((dv, lr), f_out)

## Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [0]:
df_feb.head()

In [0]:
df_feb["duration"] = df_feb["tpep_dropoff_datetime"] - df_feb["tpep_pickup_datetime"]
df_feb["duration"] = df_feb["duration"].apply(lambda td: td.total_seconds() / 60)

In [0]:
df_feb = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]
categorical = ['PULocationID', 'DOLocationID']
df_feb[categorical] = df_feb[categorical].astype(str)

In [0]:
#with open('models/lin_reg.bin', 'rb') as f:
#    dv, lr = pickle.load(f)

In [0]:
val_dicts = df_feb[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = df_feb[target].values

y_pred = lr.predict(X_val)

In [0]:
np.sqrt(mean_squared_error(y_val, y_pred))